In [7]:
import os
import json
import glob
import psycopg2
import pandas as pd
from google.cloud import bigquery

In [26]:
filepath = 'github_events_01.json'
all_files = []
for root, dirs, files in os.walk(filepath):
    files = glob.glob(os.path.join(root, "*.json"))
    for f in files:
        all_files.append(os.path.abspath(f))

num_files = len(all_files)
print(f"{num_files} files found in {filepath}")

0 files found in github_events_01.json


In [25]:




def process(cur, conn, filepath):
    # Get list of files from filepath
    all_files = get_files(filepath)

    for datafile in all_files:
        with open(datafile, "r") as f:
            data = json.loads(f.read())
            for each in data:            

                # Insert data into tables here
                insert_statement = f"""
                    INSERT INTO events (
                        id,
                        type,
                        actor_id
                    ) VALUES ('{each["id"]}', '{each["type"]}', '{each["actor"]["id"]}')
                    ON CONFLICT (id) DO NOTHING
                """
                cur.execute(insert_statement)

                insert_statement = f"""
                    INSERT INTO repo (
                        id,
                        name,
                        url,
                        actor_id
                    ) VALUES (
                        '{each["repo"]["id"]}',
                        '{each["repo"]["name"]}',
                        '{each["repo"]["url"]}',
                        '{each["actor"]["id"]}'
                    ) ON CONFLICT (id) DO NOTHING
                """


                conn.commit()

In [ ]:
def main():
    conn = psycopg2.connect(
        "host=127.0.0.1 dbname=postgres user=postgres password=postgres"
    )
    cur = conn.cursor()

    process(cur, conn, filepath="../data")

    conn.close()


if __name__ == "__main__":
    main()

In [ ]:


# Define your schema
schema = [
    bigquery.SchemaField("first_name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("last_name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("dob", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("age", "INTEGER", mode="REQUIRED")
]

# Specify the table ID
table_id = "your_project.your_dataset.your_table"

# Create a BigQuery client
client = bigquery.Client()

# Create the table
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)

# Print confirmation message
print("Created Table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))
